In [59]:
import os
from google.cloud import language_v1
import pandas as pd
from cleantext import clean
import json
import glob
from pprint import pprint
import datetime
import re 
import glob
import nltk  
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from tqdm import tqdm
nltk.download()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ashlyn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [60]:

dollar_bitcoin_df = pd.read_csv("../Resources/dollar_bitcoin_1.csv",lineterminator='\n')
for i in range(1, 15):
    df = pd.read_csv(f"../Resources/dollar_bitcoin_{i}.csv",lineterminator='\n')
    dollar_bitcoin_df = pd.concat([dollar_bitcoin_df, df], axis=0)
    
dollar_bitcoin_df.head()


,Unnamed: 0,Datetime,Text
0,0,2022-05-01 23:59:56+00:00,Oh you think im just pulling this out my ass!?...
1,1,2022-05-01 23:59:56+00:00,@Cryptic_Oz Bitcoin es baloncesto! https://t.c...
2,2,2022-05-01 23:59:53+00:00,UN LONG A SANDBOX !! 💥#sandbox #crypto #bitcoi...
3,3,2022-05-01 23:59:52+00:00,I’ve realized I’m not attracted to religions w...
4,4,2022-05-01 23:59:50+00:00,This is ridiculous. Those are stocks! \n\nBitc...


In [61]:
dollar_bitcoin_df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))


,Unnamed: 0,Datetime,Text
0,0,2022-05-01 23:59:56+00:00,Oh you think im just pulling this out my ass!?...
1,1,2022-05-01 23:59:56+00:00,@Cryptic_Oz Bitcoin es baloncesto! https://t.c...
2,2,2022-05-01 23:59:53+00:00,UN LONG A SANDBOX !! #sandbox #crypto #bitcoin...
3,3,2022-05-01 23:59:52+00:00,Ive realized Im not attracted to religions whe...
4,4,2022-05-01 23:59:50+00:00,This is ridiculous. Those are stocks! \n\nBitc...
...,...,...,...
9996,9996,2022-05-14 19:53:35+00:00,ma mre a investi dans le bitcoin. Fin du tweet
9997,9997,2022-05-14 19:53:32+00:00,@crypto_gateway Purtroppo che Bitcoin come I...
9998,9998,2022-05-14 19:53:31+00:00,New piece on how #Bitcoin mining encourages re...
9999,9999,2022-05-14 19:53:26+00:00,#Bitcoin Last Price $29343 #BTC \nDaily Indica...


In [62]:
dollar_bitcoin_df.to_csv('dollar_bitcoin.csv')

In [63]:
def tweet_to_words(tweet):
    text = tweet.lower()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    words = text.split()
    words = [w for w in words if w not in stopwords.words("english")]
    words = [PorterStemmer().stem(w) for w in words]
    return words

In [64]:
cleantext=[]
for item in tqdm(df['Text']):
    words=tweet_to_words(item)
    cleantext+=[words]
df['cleaned']=cleantext
df

100%|████████████████████████████████████| 10001/10001 [01:06<00:00, 151.10it/s]


,Unnamed: 0,Datetime,Text,cleaned
0,0,2022-05-14 23:59:58+00:00,@mineyour_biz @notsofast I understand the nega...,"[mineyour, biz, notsofast, understand, neg, ke..."
1,1,2022-05-14 23:59:57+00:00,@Sayter @carlquintanilla @WSJ The #Bitcoin did...,"[sayter, carlquintanilla, wsj, bitcoin, chang,..."
2,2,2022-05-14 23:59:54+00:00,So how long until we get the big #Bitcoin liqu...,"[long, get, big, bitcoin, liquid, know, one, t..."
3,3,2022-05-14 23:59:50+00:00,#Bitcoin #cro #BNB enjoy the ups and the downs...,"[bitcoin, cro, bnb, enjoy, up, down, q, circl]"
4,4,2022-05-14 23:59:49+00:00,@AdamKinzinger It's any age to buy a gun in th...,"[adamkinzing, age, buy, gun, park, lot, gun, s..."
...,...,...,...,...
9996,9996,2022-05-14 19:53:35+00:00,ma mère a investi dans le bitcoin. Fin du tweet,"[investi, dan, le, bitcoin, fin, du, tweet]"
9997,9997,2022-05-14 19:53:32+00:00,@crypto_gateway Purtroppo è che Bitcoin è come...,"[crypto, gateway, purtroppo, che, bitcoin, com..."
9998,9998,2022-05-14 19:53:31+00:00,New piece on how #Bitcoin mining encourages re...,"[new, piec, bitcoin, mine, encourag, renew, en..."
9999,9999,2022-05-14 19:53:26+00:00,#Bitcoin Last Price $29343 #BTC 🚀\nDaily Indic...,"[bitcoin, last, price, 29343, btc, daili, indi..."


In [52]:
def unlist(list):
    words=''
    for item in list:
        words+=item+' '
    return words

In [53]:
def compute_vader_scores(df, label):
    sid = SentimentIntensityAnalyzer()
    df["negative"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["neg"])
    df["neutral"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["neu"])
    df["positive"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["pos"])
    df["comp"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["compound"])
    return df

In [54]:
final_bitcoin = compute_vader_scores(df,'cleaned')
final_bitcoin

,Unnamed: 0,Datetime,Text,cleaned,negative,neutral,positive,comp
0,0,2022-05-14 23:59:58+00:00,@mineyour_biz @notsofast I understand the nega...,"[mineyour, biz, notsofast, understand, neg, ke...",0.000,0.833,0.167,0.3818
1,1,2022-05-14 23:59:57+00:00,@Sayter @carlquintanilla @WSJ The #Bitcoin did...,"[sayter, carlquintanilla, wsj, bitcoin, chang,...",0.199,0.554,0.247,0.2263
2,2,2022-05-14 23:59:54+00:00,So how long until we get the big #Bitcoin liqu...,"[long, get, big, bitcoin, liquid, know, one, t...",0.000,1.000,0.000,0.0000
3,3,2022-05-14 23:59:50+00:00,#Bitcoin #cro #BNB enjoy the ups and the downs...,"[bitcoin, cro, bnb, enjoy, up, down, q, circl]",0.000,0.652,0.348,0.4939
4,4,2022-05-14 23:59:49+00:00,@AdamKinzinger It's any age to buy a gun in th...,"[adamkinzing, age, buy, gun, park, lot, gun, s...",0.276,0.575,0.149,-0.2960
...,...,...,...,...,...,...,...,...
9996,9996,2022-05-14 19:53:35+00:00,ma mère a investi dans le bitcoin. Fin du tweet,"[investi, dan, le, bitcoin, fin, du, tweet]",0.000,1.000,0.000,0.0000
9997,9997,2022-05-14 19:53:32+00:00,@crypto_gateway Purtroppo è che Bitcoin è come...,"[crypto, gateway, purtroppo, che, bitcoin, com...",0.000,0.934,0.066,0.3400
9998,9998,2022-05-14 19:53:31+00:00,New piece on how #Bitcoin mining encourages re...,"[new, piec, bitcoin, mine, encourag, renew, en...",0.000,1.000,0.000,0.0000
9999,9999,2022-05-14 19:53:26+00:00,#Bitcoin Last Price $29343 #BTC 🚀\nDaily Indic...,"[bitcoin, last, price, 29343, btc, daili, indi...",0.055,0.945,0.000,-0.2960


In [55]:
# print(len(date), len(final_bitcoin.cleaned),len(final_bitcoin.negative),len(final_bitcoin.comp),len(final_bitcoin.neutral))

In [56]:
# final_bitcoin.isnull().values.any()

final_bitcoin[['dates','Time']] = final_bitcoin.Datetime.str.split("+",expand=True)

final_bitcoin
               


,Unnamed: 0,Datetime,Text,cleaned,negative,neutral,positive,comp,dates,Time
0,0,2022-05-14 23:59:58+00:00,@mineyour_biz @notsofast I understand the nega...,"[mineyour, biz, notsofast, understand, neg, ke...",0.000,0.833,0.167,0.3818,2022-05-14 23:59:58,00:00
1,1,2022-05-14 23:59:57+00:00,@Sayter @carlquintanilla @WSJ The #Bitcoin did...,"[sayter, carlquintanilla, wsj, bitcoin, chang,...",0.199,0.554,0.247,0.2263,2022-05-14 23:59:57,00:00
2,2,2022-05-14 23:59:54+00:00,So how long until we get the big #Bitcoin liqu...,"[long, get, big, bitcoin, liquid, know, one, t...",0.000,1.000,0.000,0.0000,2022-05-14 23:59:54,00:00
3,3,2022-05-14 23:59:50+00:00,#Bitcoin #cro #BNB enjoy the ups and the downs...,"[bitcoin, cro, bnb, enjoy, up, down, q, circl]",0.000,0.652,0.348,0.4939,2022-05-14 23:59:50,00:00
4,4,2022-05-14 23:59:49+00:00,@AdamKinzinger It's any age to buy a gun in th...,"[adamkinzing, age, buy, gun, park, lot, gun, s...",0.276,0.575,0.149,-0.2960,2022-05-14 23:59:49,00:00
...,...,...,...,...,...,...,...,...,...,...
9996,9996,2022-05-14 19:53:35+00:00,ma mère a investi dans le bitcoin. Fin du tweet,"[investi, dan, le, bitcoin, fin, du, tweet]",0.000,1.000,0.000,0.0000,2022-05-14 19:53:35,00:00
9997,9997,2022-05-14 19:53:32+00:00,@crypto_gateway Purtroppo è che Bitcoin è come...,"[crypto, gateway, purtroppo, che, bitcoin, com...",0.000,0.934,0.066,0.3400,2022-05-14 19:53:32,00:00
9998,9998,2022-05-14 19:53:31+00:00,New piece on how #Bitcoin mining encourages re...,"[new, piec, bitcoin, mine, encourag, renew, en...",0.000,1.000,0.000,0.0000,2022-05-14 19:53:31,00:00
9999,9999,2022-05-14 19:53:26+00:00,#Bitcoin Last Price $29343 #BTC 🚀\nDaily Indic...,"[bitcoin, last, price, 29343, btc, daili, indi...",0.055,0.945,0.000,-0.2960,2022-05-14 19:53:26,00:00


In [57]:
final_bitcoin = final_bitcoin.filter(['dates', 'cleaned','negative','neutral','positive','comp'])
final_bitcoin.head(40)

,dates,cleaned,negative,neutral,positive,comp
0,2022-05-14 23:59:58,"[mineyour, biz, notsofast, understand, neg, ke...",0.000,0.833,0.167,0.3818
1,2022-05-14 23:59:57,"[sayter, carlquintanilla, wsj, bitcoin, chang,...",0.199,0.554,0.247,0.2263
2,2022-05-14 23:59:54,"[long, get, big, bitcoin, liquid, know, one, t...",0.000,1.000,0.000,0.0000
3,2022-05-14 23:59:50,"[bitcoin, cro, bnb, enjoy, up, down, q, circl]",0.000,0.652,0.348,0.4939
4,2022-05-14 23:59:49,"[adamkinzing, age, buy, gun, park, lot, gun, s...",0.276,0.575,0.149,-0.2960
5,2022-05-14 23:59:49,"[dreahumphrey, rebelnewsonlin, leslynlewi, bit...",0.000,0.854,0.146,0.4404
6,2022-05-14 23:59:49,"[updat, bitcoin, transact, fee, bch, next, blo...",0.244,0.756,0.000,-0.7003
7,2022-05-14 23:59:47,"[el, salvador, adopt, bitcoin, 7, 09, 2021, su...",0.000,0.855,0.145,0.3400
8,2022-05-14 23:59:46,"[elonmusk, almost, everon, known, sinc, start,...",0.120,0.688,0.192,0.2500
9,2022-05-14 23:59:41,"[bitcoin, one, exampl, someth, use, blockchain...",0.000,1.000,0.000,0.0000
